# FDA Reading Room 483 Analysis

In [1]:
from datetime import date
print("Script last ran on {}".format(date.today().strftime("%m/%d/%Y")))

Script last ran on 05/04/2021


In [2]:
# import packages
import numpy as np
import pandas as pd
import requests
import lxml
import html5lib 
from bs4 import BeautifulSoup
import janitor

# selenium functions
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #allow  you to enter keystrokes into fields
from selenium.webdriver.support.ui import Select #allow you to select a dropdown item
from selenium.webdriver.support.ui import WebDriverWait #lets you modify a field before proceeding
from selenium.common.exceptions import NoSuchElementException 

## Step 1: Scrape Data Table and List of URLs to Form 483s from URL

__Establishment Types to Filter By__  
Outsourcing Facility, Producer of Sterile and Non Sterile Drug Products, Producer of Sterile Drug Products, Producer of Non Sterile Drug Products, Manufacturer

In [3]:
url = 'https://www.fda.gov/about-fda/office-regulatory-affairs/ora-foia-electronic-reading-room'
driver = webdriver.Chrome()
driver.get(url)

table class = lcsd-datatable--ora-foia-reading table table-bordered dataTable no footer dtr-inline

In [4]:
# utilize the filter box to select only 483s (value = 0)
select = Select(driver.find_element_by_css_selector('#lcds-datatable-filter--record-filter'))
select.select_by_value('0')

# get the table headers
## read the datatable of page 1 for the sake of retrieving headers
datatable_xpath = '//*[@id="DataTables_Table_0"]'
records = driver.find_element_by_xpath(datatable_xpath)
records_innerhtml = records.get_attribute('innerHTML')
soup = BeautifulSoup(records_innerhtml, 'html.parser')
trs = soup.findAll('tr')
headers = []
for th in trs[0].findAll('th'):
    headers.append(th.text)
headers

# scrape all rows in each page of the paginated datatable
final_page = int(driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/ul/li[8]').text)
rows = []
for page in range(1, final_page + 1):
    
    datatable_xpath = '//*[@id="DataTables_Table_0"]'
    records = driver.find_element_by_xpath(datatable_xpath)
    records_innerhtml = records.get_attribute('innerHTML')
    soup = BeautifulSoup(records_innerhtml, 'html.parser')
    trs = soup.findAll('tr')
    
    for i in range(1, len(trs)):
        tds = []
        #page_rows = []
        for td in trs[i].findAll('td'):
            a = td.findAll('a')
            spans = td.findAll('span')
            inputs = td.findAll('input')
            ret = ""
            if len(a) != 0 or len(spans) != 0 or len(inputs) != 0:
                if len(a) != 0:
                    for link in a:
                        ret += link.text + ' - '+link['href']
                if len(spans) != 0:
                    for span in spans:
                        ret += span.text + ' - '+span['title']
                if len(inputs) != 0:
                    for inp in inputs:
                        if inp.has_attr('value'):
                            if inp.has_attr('type'):
                                if inp['type'] == 'hidden':
                                    ret += inp['value']
            else: 
                ret = td.text if td.text != '' and td.text != '\n' else "NaN"
            tds.append(ret)
        rows.append(tds)
        
    driver.find_element_by_xpath('//*[@id="DataTables_Table_0_next"]/a').click()

len(rows)

1463

In [5]:
df = pd.DataFrame(rows, columns = headers)
df[['Record Type', 'HREF']] = df['Record Type'].str.split(' - ', 1, expand = True)
df = df.clean_names()
df['pdf_url'] = 'https://www.fda.gov' + df['href']
df.head()

,record_date,fei_number,company_name,record_type,state,establishment_type,publish_date,href,pdf_url
0,04/20/2021,3015448605,"Emergent Manufacturing Operations Baltimore, LLC.",483,Maryland,Vaccine Manufacturer,04/21/2021,/media/147762/download,https://www.fda.gov/media/147762/download
1,02/05/2021,3006701521,Midwestern Pet Foods Chickasha Operation,483,Oklahoma,Pet Food Manufacturer,04/19/2021,/media/146854/download,https://www.fda.gov/media/146854/download
2,01/28/2021,3011761321,"Wells Pharmacy, Inc",483,Tennessee,Outsourcing Facility,03/31/2021,/media/147184/download,https://www.fda.gov/media/147184/download
3,01/05/2021,3011834594,"Juno Therapeutics, Inc.",483 Response,Washington,CAR-T Cell Therapy Manufacturer,01/25/2021,/media/145363/download,https://www.fda.gov/media/145363/download
4,12/10/2020,3013629214,"Lonza Houston, Inc",483,Texas,Manufacturer,01/14/2021,/media/145127/download,https://www.fda.gov/media/145127/download


## Step 2: Read each 483 PDF through an OCR machine into notebook

In [6]:
list_urls = list(df.pdf_url)
test_pdf = list_urls[0]

In [7]:
driver.quit()

## Step 3: Tidy each document into a corpus

## Step 4: Perform Topic Modeling on Most Recent 1 Year of Drug Observations

## References

__Useful Links__  
https://medium.com/@sarfrazarshad/scraping-dynamically-created-tables-196b7cbe6c84  
https://stackoverflow.com/questions/56757261/extract-href-using-pandas-read-html/56757977  
https://stackoverflow.com/questions/60757571/python-pandas-parse-html-table-to-get-hidden-values-and-links  
https://stackoverflow.com/questions/51092362/selenium-clicking-to-next-page-until-on-last-page